<a href="https://colab.research.google.com/github/edsondearaujo/correlation_analysis/blob/main/correlation_analisys_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Como testar a aplicação no Jupyter Notebook

Este notebook foi dividido em várias células para facilitar a execução e compreensão das etapas do código. Aqui está um passo a passo para testar o projeto:

### Passo 1: Executando as células
Para rodar o código de cada célula, basta seguir os seguintes passos:

1. **Selecione a célula** com o código que deseja executar clicando nela.
2. **Pressione `Shift + Enter`** ou clique no botão de "Play" no topo da interface do Jupyter Notebook.
   - Isso executará o código da célula e irá avançar para a próxima célula automaticamente.
3. Para executar todas as células de uma vez, vá ao menu **"Cell"** e clique em **"Run All"**.

### Passo 2: Estrutura do notebook
Este notebook está organizado em etapas. Aqui está uma visão geral de cada uma:

1. **Instalação das dependências**:
   - Esta célula instala todas as bibliotecas necessárias para rodar o notebook, como `pandas`, `matplotlib`, `pyarrow`, entre outras.
   - Execute esta célula primeiro para garantir que todas as dependências estejam instaladas.

2. **Imports**:
   - Nesta célula, importamos todas as bibliotecas que serão utilizadas no código.
   - Execute esta célula após a instalação das dependências.

3. **Módulos de processamento**:
   - As células seguintes contêm as funções responsáveis por carregar, processar, mesclar os dados, e visualizá-los em gráficos.
   - Cada célula tem uma função específica que pode ser executada individualmente ou como parte do fluxo completo do código.

4. **Main**:
   - A célula com a função `main()` é a última a ser executada, e ela orquestra a execução de todo o fluxo de trabalho, desde o carregamento dos dados até a visualização dos gráficos.

### Passo 5: Visualizando os gráficos
Após rodar a função `main()` e processar seus dados, o notebook irá gerar gráficos que ajudam a visualizar a correlação entre diferentes métricas da sua planilha.



In [ ]:
# dependências
%pip install pandas matplotlib seaborn openpyxl

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

In [ ]:
#data_loader.py
def load_data(file_path, sheet_name):
    try:
        data = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
        print(f"Dados carregados com sucesso da aba '{sheet_name}'")
        return data
    except FileNotFoundError:
        print(f"Erro: O arquivo {file_path} não foi encontrado.")
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")

In [ ]:
#process_data.py
def process_data(google_ads_data, ad_occurrences_data):
    # Renomear colunas para consistência
    google_ads_data.columns = ['Dia', 'Cliques', 'Impr.', 'Custo', 'CPC']
    ad_occurrences_data.columns = ['data_formatada', 'Ocorrencia']

    # Converter as colunas de data para datetime
    google_ads_data['Dia'] = pd.to_datetime(google_ads_data['Dia'], errors='coerce')
    ad_occurrences_data['data_formatada'] = pd.to_datetime(ad_occurrences_data['data_formatada'], errors='coerce')

    # Checar se há valores nulos
    if google_ads_data['Dia'].isnull().sum() > 0 or ad_occurrences_data['data_formatada'].isnull().sum() > 0:
        print("Atenção: Existem datas inválidas ou nulas nos dados.")

    return google_ads_data, ad_occurrences_data

In [ ]:
def merge_data(google_ads_data, ad_occurrences_data):
    merged_data = pd.merge(
        google_ads_data,
        ad_occurrences_data,
        left_on='Dia',
        right_on='data_formatada',
        how='inner'  # Podemos ajustar o método dependendo do que você quiser visualizar
    )

    # Verificar o sucesso da mesclagem
    if merged_data.empty:
        print("Atenção: A mesclagem dos dados resultou em um DataFrame vazio. Verifique as datas.")
    else:
        print(f"Dados mesclados com sucesso. {len(merged_data)} linhas combinadas.")

    return merged_data

In [ ]:
def analyze_correlation(merged_data):
    correlation = merged_data['CPC'].corr(merged_data['Ocorrencia'])
    print(f"Correlação entre CPC e Ocorrências de Anúncios: {correlation}")
    return correlation

In [ ]:
#visualization.py

# Função para gráfico de dispersão
def plot_scatter(merged_data):
    plt.figure(figsize=(10, 6))
    plt.scatter(merged_data['Ocorrencia'], merged_data['CPC'], alpha=0.7, c='blue', edgecolors='w', s=100)

    plt.title('Dispersão entre Ocorrências de Anúncios e CPC')
    plt.xlabel('Ocorrências de Anúncios')
    plt.ylabel('CPC')
    plt.grid(True)
    plt.show()


# Função para regressão linear
def plot_regression(merged_data):
    sns.lmplot(x='Ocorrencia', y='CPC', data=merged_data, aspect=1.5, scatter_kws={'alpha': 0.7})
    plt.title('Regressão Linear: Ocorrências de Anúncios vs CPC')
    plt.grid(True)
    plt.show()

# Análise temporal do CPC
def plot_monthly_cpc_trend(google_ads_data):
    # Filtrar os dados a partir de Abril/2023
    google_ads_data_filtered = google_ads_data[google_ads_data['Dia'] >= '2023-04-01']

    # Calcular a média mensal do CPC
    monthly_cpc = google_ads_data_filtered.resample('ME', on='Dia')['CPC'].mean()

    # Plotar gráfico de linha da queda do CPC ao longo dos meses
    plt.figure(figsize=(10, 6))
    plt.plot(monthly_cpc.index, monthly_cpc.values, marker='o', linestyle='-', color='b')
    plt.title('Média Mensal do CPC (Abril 2023 em diante)')
    plt.xlabel('Mês')
    plt.ylabel('CPC Médio')
    plt.grid(True)
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo no seu Google Drive
file_path = 'https://docs.google.com/spreadsheets/d/1rcY7DCy2Cn3pAa1p0OVCOgdmbit8tkiI/edit?usp=drive_link&ouid=112527837874365698513&rtpof=true&sd=true'

# Carregar o arquivo Excel
df = pd.read_excel(file_path)
print(df.head())


In [ ]:
#main.py
def main():
    # Carrega o arquivo Excel
    file_path = '/content/cpc_data.xlsx'

    # Carregar dados de cada aba
    google_ads_data = load_data(file_path, 'Relatório Google Ads + BrandMon')
    ad_occurrences_data = load_data(file_path, 'Ocorrências de Anúncios')

    if google_ads_data is not None and ad_occurrences_data is not None:
        # Processar e limpar os dados
        google_ads_data, ad_occurrences_data = process_data(google_ads_data, ad_occurrences_data)

        # Mesclar os dados
        merged_data = merge_data(google_ads_data, ad_occurrences_data)

        if not merged_data.empty:
            # Analisar correlação
            analyze_correlation(merged_data)

            # Plotar gráfico de dispersão
            plot_scatter(merged_data)

            # Plotar a queda do CPC ao longo dos meses (de abril 2023 em diante)
            plot_monthly_cpc_trend(google_ads_data)

            # Gerar uma regressão linear para entender melhor a relação
            plot_regression(merged_data)

if __name__ == '__main__':
    main()


Erro: O arquivo /content/cpc_data.xlsx não foi encontrado.
Erro: O arquivo /content/cpc_data.xlsx não foi encontrado.
